In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LeakyReLU, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os

apsolutna putanja do podataka

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [5]:
dir = os.listdir(path)

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
percent_val = 0.2

Konkatenacija podataka koji se posle koriste za adekvatnu raspodelu na skup za treniranje i validaciju

In [9]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [10]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\xCmd.txt",header=None).astype('int'))

In [11]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

Ucitavanje stadardnih devijacija i strednih vrednosti

In [12]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [13]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Raspodela podataka kako bi se izbegla fiktivna stanja

In [16]:
for j in range(len(train_files_names)):
    exec(f"koord_trening_{j} = []")
    exec(f"razlika_trening_{j} = []")

    for i in range(61000):
        exec(f"koord_trening_{j}.append(train_[j*61000+i])")
        exec(f"razlika_trening_{j}.append(x_command[i] - train_[j*61000+i])")
    exec(f'koord_trening_{j} = np.asarray(koord_trening_{j})') 
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')  

In [15]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

# TESTIRANJE ARHITEKTURE i LR/BATCH

napravljena je for petlja za testiranje hiper-parametara/arhitekture

In [17]:
filter_size = [16,32]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
output_dens = [16,32]
#batch_size = [256]
time_step_list = [8,10]

In [18]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
#best_batch = None
best_out = None
best_step = None
loop = 0
for step in time_step_list:
    
    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_koord_X_{i}, _ = create_timeseries_split(koord_trening_{i}, step)')
        
    for i in range(len(train_files_names)):
        exec(f'_, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')
    
    train_X = np.asarray(train_koord_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_koord_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    #uzimamo percent_val sa kraja train_X skupa 
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    for filt in filter_size:
        for kernel_1 in kernel_size_1:
            for kernel_2 in kernel_size_2:
                for pool in pooling_size:
                    for out in output_dens:
                        keras.utils.set_random_seed(7)
                        optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
                        loop += 1

                        modelMSE = Sequential()
                        modelMSE.add(Input(shape=input_shape))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                        modelMSE.add(MaxPooling1D(pool_size=pool))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_2,padding='same', activation=LeakyReLU()))
                        modelMSE.add(Flatten())
                        modelMSE.add(Dense(out, activation=LeakyReLU()))
                        modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                        modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                        modelMSE.fit(train_X, train_y, epochs = 10, batch_size = 256, verbose = 2, shuffle = False)
                        test_predict_MSE = modelMSE.predict(val_x)



                        #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                        #radimo evaluaciju na mean 3 sigma testu
                        i =(len(x_command)-step) - val_x.shape[0]%(len(x_command)-step) -1
                        mean_3std = 0
                        for j in range(len(test_predict_MSE)):
                            if i%len(x_command) == 0:
                                i = step
                            if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                    mean_3std += 1
                            i += 1



                        if mean_3std > best_skor:
                            best_skor = mean_3std
                            best_filter = filt
                            best_kernel_1 = kernel_1
                            best_kernel_2 = kernel_2
                            best_pool = pool
                            #best_batch = batch
                            best_out = out
                            best_step = step


                            print('best_skor: ', best_skor)
                            print('best_filter: ', best_filter)
                            print('best_kernel_1: ', best_kernel_1)
                            print('best_kernel_2: ', best_kernel_2)
                            print('best_out: ', best_out )
                            #print('best_batch', best_batch)
                            print('best_step', best_step)
                            print('best_pool', best_pool)

                        print('loop: ', loop)

print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
print('best_out: ', best_out )
#print('best_batch', best_batch)
print('best_step', best_step)
print('best_pool', best_pool)



Epoch 1/10

3431/3431 - 9s - loss: 10710.6904 - 9s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 6s - loss: 484.0030 - 6s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 6s - loss: 537.4827 - 6s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 7s - loss: 565.4360 - 7s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 6s - loss: 492.1724 - 6s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 6s - loss: 505.8602 - 6s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 6s - loss: 466.5983 - 6s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 7s - loss: 423.3464 - 7s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 7s - loss: 400.8174 - 7s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 7s - loss: 379.6429 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
best_skor:  80796
best_filter:  16
best_kernel_1:  3
best_kernel_2:  3
best_out:  16
best_step 8
best_pool 3
loop:  1
Epoch 1/10
3431/3431 - 8s - loss: 1115.7491 - 8s/epoch - 2ms/step
Epoch 2/10
3431/3431 - 7s - loss: 1023.5324 - 7s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 7s - loss: 10

3431/3431 - 7s - loss: 130.1056 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 11s 1ms/step
loop:  10
Epoch 1/10
3431/3431 - 9s - loss: 88229.8516 - 9s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 7s - loss: 527.5104 - 7s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 8s - loss: 676.8199 - 8s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 8s - loss: 558.9288 - 8s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 7s - loss: 674.8254 - 7s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 7s - loss: 604.4867 - 7s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 7s - loss: 480.0391 - 7s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 7s - loss: 543.0250 - 7s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 7s - loss: 562.1691 - 7s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 7s - loss: 459.8090 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 11s 1ms/step
loop:  11
Epoch 1/10
3431/3431 - 9s - loss: 799.3262 - 9s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 7s - loss: 828.1263 - 7s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 7s -

Epoch 3/10
3431/3431 - 8s - loss: 2037.1686 - 8s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 8s - loss: 2282.5491 - 8s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 8s - loss: 1866.3073 - 8s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 8s - loss: 1948.5480 - 8s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 8s - loss: 1764.5774 - 8s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 8s - loss: 1393.3195 - 8s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 8s - loss: 1299.5304 - 8s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 8s - loss: 1192.6597 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 11s 1ms/step
loop:  22
Epoch 1/10
3431/3431 - 9s - loss: 2882.8096 - 9s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 9s - loss: 1655.1550 - 9s/epoch - 3ms/step
Epoch 3/10
3431/3431 - 10s - loss: 1424.1946 - 10s/epoch - 3ms/step
Epoch 4/10
3431/3431 - 10s - loss: 1392.4050 - 10s/epoch - 3ms/step
Epoch 5/10
3431/3431 - 9s - loss: 1534.6735 - 9s/epoch - 3ms/step
Epoch 6/10
3431/3431 - 8s - loss: 1444.2053 - 8s/epoch - 2ms/step
Epo

Epoch 5/10
3431/3431 - 8s - loss: 726.6694 - 8s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 8s - loss: 671.3202 - 8s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 8s - loss: 737.2707 - 8s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 8s - loss: 576.3217 - 8s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 8s - loss: 544.7385 - 8s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 8s - loss: 522.9661 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 11s 1ms/step
loop:  33
Epoch 1/10
3431/3431 - 10s - loss: 2458.9153 - 10s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 8s - loss: 1168.9922 - 8s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 8s - loss: 1191.5153 - 8s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 8s - loss: 1199.7119 - 8s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 8s - loss: 1001.3012 - 8s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 8s - loss: 1203.2628 - 8s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 8s - loss: 812.0783 - 8s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 8s - loss: 777.7948 - 8s/epoch - 2ms/step
Epoch 9/10
34

Epoch 9/10
3431/3431 - 8s - loss: 127.5097 - 8s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 8s - loss: 148.9316 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 11s 1ms/step
loop:  44
Epoch 1/10
3431/3431 - 9s - loss: 11931.4941 - 9s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 8s - loss: 2501.6892 - 8s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 8s - loss: 2143.5479 - 8s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 8s - loss: 1409.9159 - 8s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 8s - loss: 1181.0199 - 8s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 8s - loss: 909.4472 - 8s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 9s - loss: 767.3848 - 9s/epoch - 3ms/step
Epoch 8/10
3431/3431 - 8s - loss: 717.7672 - 8s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 8s - loss: 572.3372 - 8s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 8s - loss: 579.8595 - 8s/epoch - 2ms/step
9149/9149 [==============================] - 11s 1ms/step
loop:  45
Epoch 1/10
3431/3431 - 10s - loss: 869.0903 - 10s/epoch - 3ms/step
Epoch 2/1

9149/9149 [==============================] - 12s 1ms/step
loop:  55
Epoch 1/10
3431/3431 - 11s - loss: 5997.4287 - 11s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 10s - loss: 2107.5291 - 10s/epoch - 3ms/step
Epoch 3/10
3431/3431 - 10s - loss: 2041.6956 - 10s/epoch - 3ms/step
Epoch 4/10
3431/3431 - 10s - loss: 2702.5081 - 10s/epoch - 3ms/step
Epoch 5/10
3431/3431 - 10s - loss: 2307.3843 - 10s/epoch - 3ms/step
Epoch 6/10
3431/3431 - 9s - loss: 1530.8903 - 9s/epoch - 3ms/step
Epoch 7/10
3431/3431 - 9s - loss: 1641.1599 - 9s/epoch - 3ms/step
Epoch 8/10
3431/3431 - 10s - loss: 1860.4436 - 10s/epoch - 3ms/step
Epoch 9/10
3431/3431 - 10s - loss: 1331.3813 - 10s/epoch - 3ms/step
Epoch 10/10
3431/3431 - 10s - loss: 1470.6270 - 10s/epoch - 3ms/step
9149/9149 [==============================] - 13s 1ms/step
loop:  56
Epoch 1/10
3431/3431 - 12s - loss: 5678.7456 - 12s/epoch - 4ms/step
Epoch 2/10
3431/3431 - 10s - loss: 2708.1780 - 10s/epoch - 3ms/step
Epoch 3/10
3431/3431 - 10s - loss: 2614.2739 - 10s/

In [19]:
time_steps = 8
input_shape = (time_steps,1)

for i in range(len(train_files_names)):
    exec(f'train_koord_X_{i}, _ = create_timeseries_split(koord_trening_{i}, time_steps)')

for i in range(len(train_files_names)):
    exec(f'_, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')

train_X = np.asarray(train_koord_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_koord_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  

In [27]:
val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

train_X = train_X[:len(train_X)-val_x.shape[0]]
train_y = train_y[:len(train_y)-val_x.shape[0]]

train_X = train_X[:len(train_X)-val_x.shape[0]]
train_y = train_y[:len(train_y)-val_x.shape[0]]

napravljen je model na osnovu prethodnih najboljih rezultata

In [28]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=16, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Conv1D(filters=16, kernel_size=4,padding='same', activation=LeakyReLU()))
#modelMSE.add(GlobalAveragePooling1D())
modelMSE.add(Flatten())
modelMSE.add(Dense(32, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [29]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
3431/3431 - 9s - loss: 901.4750 - 9s/epoch - 3ms/step
Epoch 2/10
3431/3431 - 7s - loss: 900.1243 - 7s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 7s - loss: 872.3401 - 7s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 7s - loss: 711.1262 - 7s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 7s - loss: 692.8896 - 7s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 7s - loss: 669.7047 - 7s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 7s - loss: 750.3065 - 7s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 7s - loss: 557.1732 - 7s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 7s - loss: 523.0081 - 7s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 7s - loss: 580.2159 - 7s/epoch - 2ms/step


In [30]:
modelMSE.summary()

Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_130 (Conv1D)         (None, 8, 16)             64        
                                                                 
 max_pooling1d_65 (MaxPooli  (None, 2, 16)             0         
 ng1D)                                                           
                                                                 
 conv1d_131 (Conv1D)         (None, 2, 16)             1040      
                                                                 
 flatten_65 (Flatten)        (None, 32)                0         
                                                                 
 dense_130 (Dense)           (None, 32)                1056      
                                                                 
 dense_131 (Dense)           (None, 1)                 33        
                                                     

In [31]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 11s 1ms/step


In [32]:
i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.8410273226283556

##  jednostavnija arhitektura

testiranje jednostavnije arhitekture

In [33]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=8, kernel_size=3,padding='same', activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Flatten())
modelMSE.add(Dense(16, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [34]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
3431/3431 - 7s - loss: 100209.9062 - 7s/epoch - 2ms/step
Epoch 2/10
3431/3431 - 5s - loss: 1717.7201 - 5s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 5s - loss: 1356.0377 - 5s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 5s - loss: 1390.8787 - 5s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 5s - loss: 1367.7048 - 5s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 5s - loss: 1344.3016 - 5s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 5s - loss: 1321.3942 - 5s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 6s - loss: 1298.0878 - 6s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 5s - loss: 1274.9170 - 5s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 5s - loss: 1251.3679 - 5s/epoch - 2ms/step


In [35]:
modelMSE.summary()

Model: "sequential_67"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_132 (Conv1D)         (None, 8, 8)              32        
                                                                 
 max_pooling1d_66 (MaxPooli  (None, 2, 8)              0         
 ng1D)                                                           
                                                                 
 flatten_66 (Flatten)        (None, 16)                0         
                                                                 
 dense_132 (Dense)           (None, 16)                272       
                                                                 
 dense_133 (Dense)           (None, 1)                 17        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [36]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 10s 1ms/step


In [37]:
i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.25862392873367696

trazimo hiperparametre za jednostavniju arhitekturu

In [38]:
filter_size = [8,16]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
output_dens = [8, 16]
#batch_size = [256]

In [39]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
#best_batch = None
best_out = None
#best_step = None
loop = 0
for filt in filter_size:
    for kernel_1 in kernel_size_1:
        for kernel_2 in kernel_size_2:
            for pool in pooling_size:
                for out in output_dens:
                    keras.utils.set_random_seed(7)
                    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
                    loop += 1

                    modelMSE = Sequential()
                    modelMSE.add(Input(shape=input_shape))
                    modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                    modelMSE.add(MaxPooling1D(pool_size=pool))
                    modelMSE.add(Flatten())
                    modelMSE.add(Dense(out, activation=LeakyReLU()))
                    modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                    modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                    modelMSE.fit(train_X, train_y, epochs = 10, batch_size = 256, verbose = 2, shuffle = False)
                    test_predict_MSE = modelMSE.predict(val_x)



                    #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                    #radimo evaluaciju na mean 3 sigma testu
                    i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
                    mean_3std = 0
                    for j in range(len(test_predict_MSE)):
                        if i%len(x_command) == 0:
                            i = time_steps
                        if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                mean_3std += 1
                        i += 1



                    if mean_3std > best_skor:
                        best_skor = mean_3std
                        best_filter = filt
                        best_kernel_1 = kernel_1
                        best_kernel_2 = kernel_2
                        best_pool = pool
                        #best_batch = batch
                        best_out = out
                        #best_step = step


                        print('best_skor: ', best_skor)
                        print('best_filter: ', best_filter)
                        print('best_kernel_1: ', best_kernel_1)
                        print('best_kernel_2: ', best_kernel_2)
                        print('best_out: ', best_out )
                        #print('best_batch', best_batch)
                        #print('best_step', best_step)
                        print('best_pool', best_pool)

                    print('loop: ', loop)

print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
print('best_out: ', best_out )
#print('best_batch', best_batch)
#print('best_step', best_step)
print('best_pool', best_pool)

Epoch 1/10
3431/3431 - 7s - loss: 14727064.0000 - 7s/epoch - 2ms/step
Epoch 2/10
3431/3431 - 6s - loss: 93.9746 - 6s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 5s - loss: 189.9974 - 5s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 5s - loss: 310.5237 - 5s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 5s - loss: 342.8681 - 5s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 5s - loss: 336.3062 - 5s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 5s - loss: 385.7891 - 5s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 5s - loss: 338.4179 - 5s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 5s - loss: 324.6789 - 5s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 5s - loss: 322.9198 - 5s/epoch - 2ms/step
9149/9149 [==============================] - 9s 969us/step
best_skor:  105195
best_filter:  8
best_kernel_1:  3
best_kernel_2:  3
best_out:  8
best_pool 3
loop:  1
Epoch 1/10
3431/3431 - 6s - loss: 100209.9062 - 6s/epoch - 2ms/step
Epoch 2/10
3431/3431 - 5s - loss: 1717.7201 - 5s/epoch - 1ms/step
Epoch 3/10
3431/3431 - 5s - loss: 1356.0377 - 5

Epoch 2/10
3431/3431 - 5s - loss: 3899.8848 - 5s/epoch - 1ms/step
Epoch 3/10
3431/3431 - 5s - loss: 1642.8192 - 5s/epoch - 1ms/step
Epoch 4/10
3431/3431 - 5s - loss: 1564.6969 - 5s/epoch - 1ms/step
Epoch 5/10
3431/3431 - 5s - loss: 1534.3274 - 5s/epoch - 1ms/step
Epoch 6/10
3431/3431 - 5s - loss: 1495.4768 - 5s/epoch - 1ms/step
Epoch 7/10
3431/3431 - 5s - loss: 1462.5043 - 5s/epoch - 1ms/step
Epoch 8/10
3431/3431 - 5s - loss: 1426.8175 - 5s/epoch - 1ms/step
Epoch 9/10
3431/3431 - 5s - loss: 1382.0913 - 5s/epoch - 1ms/step
Epoch 10/10
3431/3431 - 5s - loss: 1349.7184 - 5s/epoch - 1ms/step
9149/9149 [==============================] - 9s 946us/step
loop:  12
Epoch 1/10
3431/3431 - 6s - loss: 107267.0391 - 6s/epoch - 2ms/step
Epoch 2/10
3431/3431 - 5s - loss: 134.5480 - 5s/epoch - 1ms/step
Epoch 3/10
3431/3431 - 5s - loss: 285.0579 - 5s/epoch - 1ms/step
Epoch 4/10
3431/3431 - 5s - loss: 274.8729 - 5s/epoch - 1ms/step
Epoch 5/10
3431/3431 - 5s - loss: 269.4152 - 5s/epoch - 1ms/step
Epoch 6/

Epoch 6/10
3431/3431 - 6s - loss: 147.6423 - 6s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 7s - loss: 143.3505 - 7s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 7s - loss: 158.0092 - 7s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 7s - loss: 137.8968 - 7s/epoch - 2ms/step
Epoch 10/10
3431/3431 - 7s - loss: 153.5868 - 7s/epoch - 2ms/step
9149/9149 [==============================] - 10s 1ms/step
loop:  23
Epoch 1/10
3431/3431 - 8s - loss: 11600.7021 - 8s/epoch - 2ms/step
Epoch 2/10
3431/3431 - 6s - loss: 676.8580 - 6s/epoch - 2ms/step
Epoch 3/10
3431/3431 - 7s - loss: 829.6426 - 7s/epoch - 2ms/step
Epoch 4/10
3431/3431 - 7s - loss: 651.4896 - 7s/epoch - 2ms/step
Epoch 5/10
3431/3431 - 6s - loss: 686.6376 - 6s/epoch - 2ms/step
Epoch 6/10
3431/3431 - 6s - loss: 866.6835 - 6s/epoch - 2ms/step
Epoch 7/10
3431/3431 - 6s - loss: 601.0490 - 6s/epoch - 2ms/step
Epoch 8/10
3431/3431 - 6s - loss: 830.0964 - 6s/epoch - 2ms/step
Epoch 9/10
3431/3431 - 6s - loss: 553.4227 - 6s/epoch - 2ms/step
Epoch 10/10
3431/34

In [40]:
142774/len(test_predict_MSE)

0.48768107773917974